<a href="https://colab.research.google.com/github/AeraVentis/Blur_detection/blob/main/Blure_detection_Oleg_Sharshon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  SHIFT CV WINTER 2023  
## Blurred images detection  



Идея данного ноутбука взята из "[Baseline Notebook](https://www.kaggle.com/code/amlekomtsev/baseline)" и "[Blur detection with feature Engineering](https://www.kaggle.com/code/harininarasimhan/blur-detection-with-feature-engineering/notebook)"

Судя по liderboard,  выделение агрегированных свойств изображений не самый лучший подход для классификации по наличию заблюренности. Анализируя ошибки предсказания (как FN, так и FP) видно несколько проблем такого подхода. 
1. Датасет имеет изображения в которых место расположения блюра имеет важное значение. Например, центральная часть в фокусе, а окружающие объекты (переднего или заднего плана) вне фокуса.
2. Тени вызывают ложное срабатывание классификатора, потому что имеют размытые границы по своей природе.
3. Motion blur  имеет сложную структуру контуров (я бы назвал этом "множественные четкие котуры") и сложен для детектирования.

Хотелось бы реализовать алгоритм выделения свойств контуров объектов на изображении, например, удельная размытость контуров, имеющая привязку к геометрическому расположению на фото. Наданный момент моей экспертности для этого недостаточно.

## Prepare the environment

In [ ]:
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname))

In [ ]:
#!pip install -q ipyplot

In [ ]:
import os
import numpy as np
import pandas as pd
import glob
#from ipyplot import plot_images
import cv2

import torch
from torchvision import transforms

from tqdm import tqdm
from PIL import Image
from skimage.filters import laplace, sobel, roberts

from sklearn.model_selection import train_test_split # разделение датасета
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from statsmodels.stats.outliers_influence import variance_inflation_factor  
from statsmodels.tools.tools import add_constant
from patsy import dmatrices

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# https://stackoverflow.com/questions/65827830/disabledfunctionerror-cv2-imshow-is-disabled-in-colab-because-it-causes-jupy
# патч для работы imshow в Google Colab
# from google.colab.patches import cv2_imshow

## Paths

In [ ]:
# /kaggle/input/shift-cv-winter-2023/train.csv
# /kaggle/input/shift-cv-winter-2023/test/test
# /kaggle/input/shift-cv-winter-2023/train/train

In [ ]:
train_pth = '/kaggle/input/shift-cv-winter-2023/train/train'
tr_csv_path = '/kaggle/input/shift-cv-winter-2023/train.csv'

## DataFrame

In [ ]:
df_train = pd.read_csv(tr_csv_path, dtype={'blur':int})
print(len(df_train))
# out: 2664

In [ ]:
print(df_train.head(3), df_train.shape, sep='\n')  # filename, blur

## Define

In [ ]:
convert_tensor = transforms.ToTensor()

In [ ]:
# set the work dir path

def set_dir(path):
  os.chdir(path)
  print(os.getcwd())
  file_list = os.listdir()
  return file_list

In [ ]:
# перед запуском установить рабочую директорию

def ftrs_extrctn(df):
  features=[]
  for im_pth in tqdm(df['filename']):
    gray_img2 = cv2.imread(str(im_pth),0)
    #gray_img2 = cv2.Laplacian(gray_img2, -1, ksize=5, scale=1,delta=0, borderType=cv2.BORDER_DEFAULT)
    gray_img2 = gray_img2[50:590, 50:590]


    tv_feat = calc_tv_measure(gray_img2)
    fft_feat = calc_fft_measure(gray_img2)
    lpl_feat = laplace(gray_img2)
    sbl_feat = sobel(gray_img2)
    rbt_feat = roberts(gray_img2)

    features.append([im_pth, lpl_feat.mean(),lpl_feat.var(),np.amax(lpl_feat),\
                   sbl_feat.mean(),sbl_feat.var(),np.max(sbl_feat),\
                   rbt_feat.mean(),rbt_feat.var(),np.max(rbt_feat),\
                   fft_feat, tv_feat])
  
  return features

In [ ]:
# Baseline notebook

def calc_tv_measure(gray_img):

    gray_img = convert_tensor(gray_img)

    gray_img = gray_img.detach().clone()
    gray_img = gray_img[:, gray_img.shape[1]//4:3*gray_img.shape[1]//4,gray_img.shape[2]//4:3*gray_img.shape[2]//4]
    
    w_variance = torch.sum(torch.pow(gray_img[:, :, 1:] - gray_img[:, :, :-1], 2), dim=[1, 2])
    h_variance = torch.sum(torch.pow(gray_img[:, 1:, :] - gray_img[:, :-1, :], 2), dim=[1, 2])

    l2_score = (h_variance + w_variance)

    w_variance = torch.sum(torch.abs(gray_img[:, :, 1:] - gray_img[:, :, :-1]), dim=[1, 2])
    h_variance = torch.sum(torch.abs(gray_img[:, 1:, :] - gray_img[:, :-1, :]), dim=[1, 2])
    
    l1_score = (h_variance + w_variance)
    
    tv_measure = (l1_score/l2_score).numpy()[0]
    
    return tv_measure

In [ ]:
# Baseline notebook
# https://pyimagesearch.com/2020/06/15/opencv-fast-fourier-transform-fft-for-blur-detection-in-images-and-video-streams/
# https://stackoverflow.com/questions/71677123/image-processing-for-blur-detection

def calc_fft_measure(gray_img, size=40):
    """ Определение заблюренности изображения с помощью Fourier transform:

    - Изображение переводится в частотный спектр с помощью fft, производится сдвиг начала координат и
    зануляется низкочастотный спектр.
    - Частотный спектр переводится обратно в изображение с помощью inverse fft.
    - Считается магнитуда, определяется уровень заблюренности картинки.
    Чем больше полученное значение, тем более размыто изображение
    """
    (h, w) = gray_img.shape
    (cx, cy) = (int(w / 2.0), int(h / 2.0))
    fft = np.fft.fft2(gray_img)
    fftShift = np.fft.fftshift(fft)

    fftShift[cy - size:cy + size, cx - size:cx + size] = 0
    fftShift = np.fft.ifftshift(fftShift)
    recon = np.fft.ifft2(fftShift)
    
    magnitude = 20 * np.log(np.abs(recon))
    mean = 1/np.average(magnitude)

    return mean

## Getting the features

In [ ]:
set_dir(train_pth)

In [ ]:
tr_ftrs = ftrs_extrctn(df_train)

In [ ]:
clmns=['image', 'Lpl_Mean', 'Lpl_Var', 'Lpl_Max', 'Sbl_Mean', 'Sbl_Var', 'Sbl_Max', 'Rbt_Mean', 'Rbt_Var', 'Rbt_Max', 'fft_feat', 'tv_feat']

In [ ]:
ftrs_df = pd.DataFrame(tr_ftrs, columns=clmns)
ftrs_df.head(3)

In [ ]:
# Объединение датафрейма с ответами (df_train) и 
# сгенерированных признаков (ftrs_df)

df_train = df_train.join(ftrs_df)
df_train.head(3)

In [ ]:
# Эта ячейка использовалась при работе в Google Colab, потому что, по моему субъективному мнению, платформа Kaggle не удобна. 
# 
# Запись на диск и выгрузка датафрейма для экономии времени при открытии новой сессии Google Colab
#df_train.to_pickle('/content/drive/MyDrive/Colab Notebooks/InternShipS/ShiftCFT/all_ftrs_26122022.pkl')
#df_train = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/InternShipS/ShiftCFT/all_ftrs_26122022.pkl')

In [ ]:
# Удаление из датафрейма столбцов с именами файлов

df_train = df_train.drop(['filename', 'image'], axis=1)
df_train.head(1)

In [ ]:
# Экперимент с проверкой влияния мультиколлинеарности на работу SVM
# P. S. Практически не влияет

#df_train = df_train.drop(['Sbl_Mean', 'Rbt_Var'], axis=1)

In [ ]:
# Запись на диск и выгрузка датафрейма без столбцов имён файов
# для экономии времени при открытии новой сессии Google Colab

# df_train.to_pickle('/content/drive/MyDrive/Colab Notebooks/InternShipS/ShiftCFT/all_ftrs_no_imgs_26122022.pkl')
# df_train = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/InternShipS/ShiftCFT/all_ftrs_no_imgs_26122022.pkl')

# Multicollinearity
Источник: https://pythonpip.ru/examples/obnaruzhenie-multikollinearnosti-vif-v-python

In [ ]:
# Отображение нижней части матрицы корреляции, без дублирующей верхней части и диагонали
# https://habr.com/ru/company/otus/blog/559666/

matrix = df_train.corr()
mask = np.triu(np.ones_like(matrix, dtype=bool))
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                             n=9, center="light", as_cmap=True)

plt.figure(figsize=(16, 12))

sns.heatmap(matrix, mask=mask, center=0, annot=True,
             fmt='.2f', square=True, cmap=cmap)

plt.show();

Multicollinearity-II
https://www.codecamp.ru/blog/how-to-calculate-vif-in-python/

In [ ]:
#find design matrix for linear regression model using 'rating' as response variable 
y, X = dmatrices('blur ~ Lpl_Mean+Lpl_Var+Lpl_Max+Sbl_Mean+Sbl_Var+Sbl_Max+Rbt_Mean+Rbt_Var+Rbt_Max+fft_feat+tv_feat', data=df_train, return_type='dataframe') # Rbt_Mean+Rbt_Var+Rbt_Max+

#calculate VIF for each explanatory variable
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns

#view VIF for each explanatory variable 
vif

# Prepare Train & Test DF

## PCA

In [ ]:
# pca = PCA().fit(df_train)
# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# plt.xlabel('number of components')
# plt.ylabel('cumulative explained variance');

In [ ]:
# PCA не "сработало", лучший результат 
# 0.8836772983114447 при C=400,kernel='rbf'

# pca = PCA(n_components=4)
# df_train_pca = pca.fit_transform(X_train)

## Prepare Train & Test DF

In [ ]:
# Подготовка набора данных для обучения и проверки точности

# Копирование   столбца ответов (blur)
y_train = df_train['blur']
#y_test = test[:, 0]

# Удаление первого столбца с номерами букв
X_train = df_train.loc[:, 'Lpl_Mean':]
#X_test = test[:, 1:]

In [ ]:
# Подготовка полного набора для обучения и послед применения для предсказания на тестовом наборе

X_tr_tr, y_tr_tr = X_train, y_train

In [ ]:
# Разделение набора данных на тренировочный и проверочный наборы в пропорции 80%/20%
# Без перемешивания

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

In [ ]:
print('X_train = ', X_train.shape, '\n', 'X_valid=', X_valid.shape, '\n', 'y_train = ', y_train.shape, '\n', 'y_valid = ', y_valid.shape)

In [ ]:
# Проверка работы классификатора на наборе без нормализации данных
# 

c_svm = SVC(gamma='auto') 
c_svm.fit(X_train, y_train)

pred_svc = c_svm.predict(X_valid)
accuracy_score(y_valid, pred_svc)

In [ ]:
# На нормализованном наборе с параметрами ядра SVC(kernel='poly', degree=3)) было получено 0.8311444652908068
# 0.9568 на 7 фичах "Lpl_Mean Lpl_Var Lpl_Max Sbl_Var Sbl_Max Rbt_Mean fft_feat"
# но судя по результату на kaggle это overfitting

svm_clf = make_pipeline(StandardScaler(), SVC(C=2150,kernel='rbf'))
svm_clf.fit(X_train, y_train)

pred_svc = svm_clf.predict(X_valid)
accuracy_score(y_valid, pred_svc)

In [ ]:
# PCA

# svm_clf = make_pipeline(StandardScaler(), SVC(C=400,kernel='rbf'))
# svm_clf.fit(X_train, y_train)

# pred_svc = svm_clf.predict(X_valid)
# accuracy_score(y_valid, pred_svc)

In [ ]:
# pred_svc = svm_clf.predict(X_valid)
# accuracy_score(y_valid, pred_svc)

## Test prediction

In [ ]:
svm_clf = make_pipeline(StandardScaler(), SVC(C=2150,kernel='rbf', probability=True))
svm_clf.fit(X_tr_tr, y_tr_tr)  # 

In [ ]:
test_pth = '/kaggle/input/shift-cv-winter-2023/test/test'
os.chdir(test_pth)
print('Current folder is: ', os.getcwd())
tst_file_list = os.listdir()
print('A sample of filename: ', tst_file_list[10])

In [ ]:
df_test = pd.DataFrame(tst_file_list, columns=['filename'])
df_test.head(3)

In [ ]:
feat_tst = []

In [ ]:
for im_pth in tqdm(df_test['filename']):
  gray_img2 = cv2.imread(str(im_pth),0)
  
  tv_feat = calc_tv_measure(gray_img2)
  fft_feat = calc_fft_measure(gray_img2)
  lpl_feat = laplace(gray_img2)
  sbl_feat = sobel(gray_img2)
  rbt_feat = roberts(gray_img2)
  
  feat_tst.append([im_pth, lpl_feat.mean(),lpl_feat.var(),np.amax(lpl_feat),\
                   sbl_feat.mean(),sbl_feat.var(),np.max(sbl_feat),\
                   rbt_feat.mean(),rbt_feat.var(),np.max(rbt_feat),\
                   fft_feat, tv_feat])

In [ ]:
clmns=['image', 'Lpl_Mean', 'Lpl_Var', 'Lpl_Max', 'Sbl_Mean', 'Sbl_Var', 'Sbl_Max', 'Rbt_Mean', 'Rbt_Var', 'Rbt_Max', 'fft_feat', 'tv_feat']

In [ ]:
ftrs_df_tst = pd.DataFrame(feat_tst, columns=clmns)
ftrs_df_tst.head(1)

In [ ]:
df_tst_no_img = ftrs_df_tst.drop(['image'], axis=1)
df_tst_no_img.head(3)

In [ ]:
pred_ProB_tst = svm_clf.predict_proba(df_tst_no_img)[:,1]

In [ ]:
pred_ProB_tst = np.round(pred_ProB_tst, 1)

In [ ]:
df_prob = pd.DataFrame(pred_ProB_tst)
df_prob.head(3)

In [ ]:
df_submission = pd.concat([ftrs_df_tst['image'], df_prob[0]], axis=1, keys=['filename', 'blur'])

In [ ]:
df_submission.to_csv('/kaggle/working/subm8.csv', index=False)